# Replica Exchange - Enveloping Distribution Sampling 


In [ ]:
import reeds
help(reeds)

In [ ]:
#IMPORTs: 
import os, glob
from IPython.display import HTML, Image,Markdown

from pygromos.utils import bash
from pygromos.euler_submissions import FileManager as fM

# Global Definition

In [ ]:

import os
from pygromos.utils import bash
from pygromos.euler_submissions import FileManager as fM

#needs to be provided via gromos compiling
gromosXX_bin = "/cluster/home/bschroed/gromos/gromos_native/gromosXX_mpi/bin"
gromosPP_bin = "/cluster/home/bschroed/gromos/gromos_native/gromos++/bin"
ene_ana_lib ="/cluster/home/bschroed/gromos/ene_ana_libs/ene_ana.md.lib"

#System Dependent settings:
name = "MY_NAME"
root_dir = os.getcwd()
input_folder =    root_dir+"/input/"

print(root_dir)


##input Files
###general Templates
in_template_md_imd = root_dir+"/input/template_md.imd"
in_template_reeds_imd = root_dir+"/input/template_reeds.imd"

###System dependent Files
in_cnf_file =     input_folder+"/PNMT_9lig_water.cnf"
in_top_file =     input_folder+"/PNMT_9lig_water.top"
in_pert_file =    input_folder+"/PNMT_9lig_water.ptp"
in_disres_file =  input_folder+"/PNMT_9lig_water_disres.dat"



# Optimize Single EDS-End states

In [ ]:
from reeds.modules import do_RE_EDS_generateOptimizedStates as optimizeStates 
help(optimizeStates)

In [ ]:
#Setup Simulation Step:

out_gOptStates_dir = root_dir+"/a_optimizedState"
in_name = name+"_optimize_single_state"

out_gOptStates_dir = bash.make_folder(out_gOptStates_dir)


topology_state_opt = fM.Topology(top_path=in_top_file, 
                                 disres_path=in_disres_file, 
                                 perturbation_path=in_pert_file)

system_state_opt = fM.System(coordinates=in_cnf_file, name=in_name, top=topology_state_opt)

print(system_state_opt)

## Job Execution

In [ ]:
optimizeStates.do(in_simSystem=system_state_opt, in_imd_template_path=in_template_md_imd, 
                  out_root_dir=out_gOptStates_dir, in_gromosXX_bin_dir=gromosXX_bin)

## Analysis

### Check optimized states

In [ ]:
Image(out_gOptStates_dir+"/analysis/plots/sampling_minstate_matrix.png", width=300)

# Find lower Bound

In [ ]:
from reeds.modules import do_RE_EDS_findLowerBound as findLowerBound
help(findLowerBound)

In [ ]:
out_lowerBound_dir = root_dir+"/b_lowerBound"
in_name = name+"_find_lower_bound"

out_lowerBound_dir = bash.make_folder(out_lowerBound_dir)


topology_lower_bound = fM.Topology(top_path=in_top_file, disres_path=in_disres_file, perturbation_path=in_pert_file)
system_lower_bound = fM.System(coordinates=in_cnf_file, name=in_name, top=topology_lower_bound)
print(system_lower_bound)

## Job Execution

In [ ]:
findLowerBound.do(in_simSystem=system_lower_bound, template_imd=in_template_md_imd, 
                  out_root_dir=out_lowerBound_dir, 
                  gromosXX_bin=gromosXX_bin)

## Analysis

### Verify undersampling

In [ ]:
Image(out_lowerBound_dir+"/analysis/plots/sampling_undersample_matrix.png", width=300)

# Energy Offset Estimation

In [ ]:
from reeds.modules import do_RE_EDS_eoffEstimation as eoffEstm
help(eoffEstm)

In [ ]:
out_eoff_dir = root_dir+"/c_eoff"
next_lowerBound_dir = root_dir+"/b_lower_bound/analysis/next"
in_name = name+"_energy_offsets"


out_eoff_dir = bash.make_folder(out_eoff_dir)


    
topology = fM.Topology(top_path=in_top_file, disres_path=in_disres_file, perturbation_path=in_pert_file)
system = fM.System(coordinates=glob.glob(next_lowerBound_dir+"/*.cnf"), name=in_name, top=topology)


## Job Execution

In [ ]:
last_jobID = eoffEstm.do(out_root_dir=out_eoff_dir, in_simSystem=system, 
                         in_template_imd_path=in_template_reeds_imd, 
                         in_ene_ana_lib=ene_ana_lib)

last_jobID

## Analysis

### Potential energies of the states

In [ ]:
Image(out_eoff_dir+"/analysis/plots/edsState_potential_timeseries_stageGrid_s15.png", width=600)

### Check undersampling of the replicas

In [ ]:
Image(out_eoff_dir+"/analysis/plots/sampling_undersample_matrix.png", width=300)

### Energy offset Estimation

In [ ]:
Markdown(out_eoff_dir+"/analysis/Eoff_estimate.out")

In [ ]:
Image(out_eoff_dir+"/analysis/plots/PNMT_9ligs_water_energy_offsets_eoff_s.png", width=600)

### Bottle neck

In [ ]:
Image(out_eoff_dir+"/analysis/plots/transitions.png", width=600)

# s-Optimization

In [ ]:
from reeds.modules import do_RE_EDS_sOptimisation as sOptimization
help(sOptimization)

In [ ]:
out_sopt_dir = root_dir+"/d_sopt"
next_sopt_dir = root_dir+"/c_eoff/analysis/next"
name = "PNMT_9ligs_water_sopt"

soptIterations =4

in_template_reeds_imd = glob.glob(next_sopt_dir+"/*imd")[0]

out_sopt_dir = bash.make_folder(out_sopt_dir)

        
topology = fM.Topology(top_path=in_top_file, disres_path=in_disres_file, perturbation_path=in_pert_file)
system_sopt = fM.System(coordinates=glob.glob(next_sopt_dir+"/*cnf"), name=name, top=topology)
print(system)

## Job Execution

In [ ]:
last_jobID = sOptimization.do(out_root_dir=out_sopt_dir, in_simSystem=system, 
                              in_template_imd=in_template_reeds_imd, 
                              in_ene_ana_lib_path=ene_ana_lib, 
                              soptIterations=soptIterations)


## Analysis

### Increase Replica Exchanges with N-LRTO
#### sopt1

In [ ]:
Image(out_sopt_dir+"/sopt1/analysis/plots/transitions.png", width=600)

In [ ]:
Markdown(out_sopt_dir+"/sopt1/analysis/s_opt_NLRTO.out")

#### sopt2

In [ ]:
Image(out_sopt_dir+"/sopt2/analysis/plots/transitions.png", width=600)

In [ ]:
Markdown(out_sopt_dir+"/sopt2/analysis/s_opt_NLRTO.out")

#### sopt3 

In [ ]:
Image(out_sopt_dir+"/sopt3/analysis/plots/transitions.png", width=600)

In [ ]:
Markdown(out_sopt_dir+"/sopt3/analysis/s_opt_NLRTO.out")

#### sopt4

In [ ]:
Image(out_sopt_dir+"/sopt4/analysis/plots/transitions.png", width=600)

### Check state switches in the replicas:

In [ ]:
Image(out_sopt_dir+"/sopt4/analysis/plots/transitions_trace_19.png", width=600)

In [ ]:
Image(out_sopt_dir+"/sopt4/analysis/plots/transitions_trace_14.png", width=600)

# Production Run

In [ ]:
from reeds.modules import do_RE_EDS_production as production
help(production)

In [ ]:
out_production_dir = root_dir+"/test_e_production"
next_production_dir = root_dir+"/0_input/3_next_prod"

name = "PNMT_9ligs_water_prod"

    
out_production_dir = bash.make_folder(out_production_dir)


production_imd = glob.glob(next_production_dir+"/*imd")[0]    
topology = fM.Topology(top_path=in_top_file, disres_path=in_disres_file, perturbation_path=in_pert_file)
system = fM.System(coordinates=glob.glob(next_production_dir+"/*cnf"), name=name, top=topology)

print(system)

## Job Execution

In [ ]:
last_jobID = production.do(out_root_dir=out_production_dir, 
                           in_simSystem=system, in_template_imd=production_imd, 
                           in_ene_ana_lib_path=ene_ana_lib, verbose=True)

## Analysis


### Check State Sampling

In [ ]:
Image(out_production_dir+"/analysis/plots/sampling1.png", width=600)

In [ ]:
Image(out_production_dir+"/analysis/plots/sampling1.png", width=600)

### Calculation of Free Energies

In [ ]:
Markdown(out_production_dir+"/analysis/convergence/free_energy_result.dat")

In [ ]:
Image(out_production_dir+"/analysis/convergence/PNMT_9ligs_water_prod_replica_1.png", width=600)

In [ ]:
display.